**In this notebook,**
<br>we will use flan-t5 model for question answering. the idea is as follwoing,

- we will use the model to perform multiple choise question task with no example (**zero shot**)
- the model gives output with only one word, but we need more explanation.
- we will use **few shot** learning to get the intended style of outputs

In [120]:
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, GenerationConfig, TrainingArguments, Trainer
import torch
import time
import evaluate
import pandas as pd
import numpy as np
from termcolor import colored

### Load the model
FLAN-T5 was released in the paper Scaling Instruction-Finetuned Language Models <br>
it is an enhanced version of T5 that has been finetuned in a mixture of tasks.

huggingface repo >> https://huggingface.co/google/flan-t5-base

we used torch_dtype bfloat16, developed by google brain as it is more computation efficient, details >> https://cloud.google.com/blog/products/ai-machine-learning/bfloat16-the-secret-to-high-performance-on-cloud-tpus

In [105]:
model_name='google/flan-t5-base'
original_model = AutoModelForSeq2SeqLM.from_pretrained(model_name, torch_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained(model_name)

### Load the dataset (CommonsenseQA)
***Dataset Summary***

CommonsenseQA is a new multiple-choice question answering dataset that requires different types of commonsense knowledge to predict the correct answers. It contains 12,102 questions with one correct answer and four distractor answers. 

the dataset have follwing columns:
**id, question, question_concept, choices, text and answerKey**

In [107]:
dataset_name = "commonsense_qa"
dataset = load_dataset(dataset_name)
dataset['train'][0]

Found cached dataset commonsense_qa (C:/Users/abrar/.cache/huggingface/datasets/commonsense_qa/default/1.0.0/28d68f56649a7f0c23bc68eae850af914aa03f95f810011ae8cf58cc5ff5051b)


  0%|          | 0/3 [00:00<?, ?it/s]

{'id': '075e483d21c29a511267ef62bedc0461',
 'question': 'The sanctions against the school were a punishing blow, and they seemed to what the efforts the school had made to change?',
 'question_concept': 'punishing',
 'choices': {'label': ['A', 'B', 'C', 'D', 'E'],
  'text': ['ignore', 'enforce', 'authoritarian', 'yell at', 'avoid']},
 'answerKey': 'A'}

### Function for generating prompt

In [110]:
#this function will load option for a single question
#the variable data_split can be one of the follwing: train, test, or validation.
#that means we want data from training, test or validation sets

def options(data_split,index):
    text = ''
    for i in range(len(dataset[data_split][index]['choices']['label'])):
        text = text + "-"+ dataset['train'][index]['choices']['text'][i] + '\n'
    return text

#prompt_generator generates promp
#the following cell wil show how a single prompt is generated
def prompt_generator(data_split, index):
    question = dataset[data_split][index]['question']
    text = options(data_split,index)
    answer = dataset[data_split][index]['answerKey']
    prompt = f"""{question}\n{text}\nanswer:"""
    return prompt

**let's see how a single prompt will look like,**

In [119]:
print('PROMPT>>>>>>>>')
print(prompt_generator('train', 200))

PROMPT>>>>>>>>
What are you hoping to achieve when talking to someone?
-communicating with others
-hurt feelings
-dry mouth
-intimacy
-changing behavior

answer:


In [141]:
index = 91
data_split = 'train'

#the folling line will use prompt generator function for generating prompt
#it will generate prompot from training dataset, for index 91
prompt = prompt_generator(data_split, index)


#inside tokenizer we give prompt as input, it will tokenize the inputs into smaller chunk and represent in numbers
inputs = tokenizer(prompt, return_tensors='pt')

#output is the generated response from the llm, look here original_model refers to flan-t5-base model
#the original_model fed with input['input_ids'], that are tokenized numeric representation of the prompt
#you can pring inputs['input_ids'] to see what inside them
#the generated output is within 50 words, as max_new_tokens set to 50
#skip_special_tokens = True, that means we donot want to sese noises (e.g. a special token [EOS] represents, end of sentences) 

output = tokenizer.decode(
    original_model.generate(
        inputs["input_ids"], 
        max_new_tokens=50,
    )[0], 
    skip_special_tokens=True
)
print('PROMPT')
print('---------------------------------------------------')
print(prompt)
print('---------------------------------------------------')
print()
print('MODEL GENERATED>> ', end = "")
print(colored(output, 'green'))
print('Actual answer:', dataset['train'][index]['answerKey'])
print(colored('N:B: options are as follows: first option is A, then B, C and so on.. so the answer is correct', 'red'))

PROMPT
---------------------------------------------------
What is someone who gets angry after getting drunk likely to participate in?
-violence
-falling down
-vomiting
-vomiting
-lower standards

answer:
---------------------------------------------------

MODEL GENERATED>> violence
Actual answer: A
N:B: options are as follows: first option is A, then B, C and so on.. so the answer is correct


If we look at the output we can see the model choosed one option from the options. But we need the model to explain the result in few sentences. A few shot inference can be helpful for the model to generate the intended output style.

### Example for few shot inference

**What is a few shot inference?** <br>

-- In few shot inference we show some examples to the model. So, the model understands what types of output we are expecting.

In [113]:
example = """
What is necessary for learning by yourself?
- exposure
- patience
- study
- cooperation
- thought

answer: study. learning by yourself opens enormous opportunity to gain knowledge. A person should study by him or herself to learn alone. When there is no one to help, you should learn by yourself.\n

What emotion does getting paid lead to?
- sorrow
- paying bills
- happiness
- frustration
- spending money

answer: happiness. as happiness is a positive emotion and when someone get paid they become happy. Money sometimes is associated to happiness, although money cannot buy happiness. But things that bring happiness can be bought by money.\n
"""

### Output generation with few shot inference

In [144]:
index = 91
data_split = 'train'
prompt =  example + prompt_generator(data_split, index)

inputs = tokenizer(prompt, return_tensors='pt')
output = tokenizer.decode(
    original_model.generate(
        inputs["input_ids"], 
        max_new_tokens=50,
    )[0], 
    skip_special_tokens=True
)

print('PROMPT')
print('---------------------------------------------------')
print(prompt)
print('---------------------------------------------------')
print()
print('MODEL GENERATED>> ', end = "")
print(colored(output, 'green'))

PROMPT
---------------------------------------------------

What is necessary for learning by yourself?
- exposure
- patience
- study
- cooperation
- thought

answer: study. learning by yourself opens enormous opportunity to gain knowledge. A person should study by him or herself to learn alone. When there is no one to help, you should learn by yourself.


What emotion does getting paid lead to?
- sorrow
- paying bills
- happiness
- frustration
- spending money

answer: happiness. as happiness is a positive emotion and when someone get paid they become happy. Money sometimes is associated to happiness, although money cannot buy happiness. But things that bring happiness can be bought by money.

What is someone who gets angry after getting drunk likely to participate in?
-violence
-falling down
-vomiting
-vomiting
-lower standards

answer:
---------------------------------------------------

MODEL GENERATED>> violence. violence is a violent activity. Someone who gets angry after getting